# Ryo's Tutorial 1d stellar structures

\begin{equation}
\frac{d}{dm} p = \frac{Gm}{4\pi r^4}
\end{equation}

The pressure $P$ is a function of some density $\rho$, entropy $S$, $\epsilon$,  ie $P(\rho, \epsilon, T, S)$

\begin{equation}
\frac{d}{dm} r = \frac{1}{4\pi r^2 \rho}
\end{equation}


The radius of the star responds to mass loss


This is our equationo of state:
\begin{align}
S   &= P/(\rho^\gamma) \\
\implies P    &= S\rho^\gamma
\end{align}

Choose some random initial values $S_0, \rho_0, P_0$


\begin{align}
\frac{dP}{dr} = \frac{-Gm\rho}{r^2}
\end{align}
\begin{align}
\frac{P_{i+1}-P_i}{r_{i+1}-r_i} = \frac{Gmr_i\rho_i}{r_i^2} \\
\implies P_{i+1} = (\frac{Gmr_i\rho_i}{r_i^2})( {r_{i+1}-r_i}) + P_i
\end{align}

\begin{align}
m = \int^{r}_{0} 4\pi r^2 \rho dr
\end{align}





In [53]:
from scipy.integrate import odeint
from scipy.interpolate import interp1d
from scipy.optimize import root
import numpy as np
from astropy.constants import G, M_sun, R_sun
import matplotlib.pyplot as plt
from scipy.integrate import quad
import pandas as pd

MSol = M_sun.cgs.value
R_sun = R_sun.cgs.value
G = G.cgs.value
pi = np.pi


def get_rho(Pi, gamma, S):
    return np.power(Pi / S, 1. / gamma)


def get_P(gamma, S, rhoi):
    return S * np.power(rhoi, gamma)


def get_Pj(Pi, mi, ri, rj, rhoi):
    dr = (rj - ri)
    dP_dr = - (G * mi * rhoi) / np.power(ri, 2)
    Pj = dP_dr * dr + Pi
    return Pj


def get_mj(mi, ri, rj, rhoi):
    dm_dr = (4. / 3.) * pi * (rj ** 3 - ri ** 3) * rhoi
    dr = (rj - ri)
    return mi + dm_dr * dr


def get_jth_values(Pi, mi, ri, rhoi, dr, S0, gamma):
    rj = ri + dr
    Pj = get_Pj(Pi, mi, ri, rj, rhoi)
    rhoj = get_rho(Pi=Pj, S=S0, gamma=gamma)
    mj = get_mj(mi, ri, rj, rhoi)
    return rj, Pj, rhoj, mj


class Star:
    def __init__(self, r, P, rho, m):
        self.r = r
        self.P = P
        self.rho = rho
        self.m = m

    @classmethod
    def build(cls, S0, rho0, gamma, m0, max_steps=int(1e6)):

        P0 = get_P(rhoi=rho0, S=S0, gamma=gamma)
        r_guess = cls.get_system_radii(P0, m0, 0.01, rho0, S0, gamma)

        r, dr = np.linspace(0.01, r_guess, 1000, retstep=True)
        P = np.zeros(len(r))
        rho = np.zeros(len(r))
        m = np.zeros(len(r))

        rho[0], m[0], P[0] = rho0, m0, P0

        for j in range(1, max_steps):
            Pi, mi, ri, rhoi = P[j - 1], m[j - 1], r[j - 1], rho[j - 1]
            _, P[j], rho[j], m[j] = get_jth_values(Pi, mi, ri, rhoi, dr, S0, gamma)

        # mask data to where rho > 0
        rmask = rho > 0
        r = r[rmask]
        P = P[rmask]
        rho = rho[rmask]
        m = m[rmask]

        # if len(r) < 10:
        #     raise ValueError("Too few points where ddensity is above 0")
        # save only 1000 points

        return cls(r, P, rho, m)

    @staticmethod
    def get_system_radii(P, m, r, rho, S0, gamma):
        dr = R_sun / 1000
        while P > 0:
            P, m, r, rho = get_jth_values(P, m, r, rho, dr, S0, gamma)
        print(r)
        return r

    def plot_profile(self):
        """plot star rho, mass, pressure"""
        plt.close('all')
        fig, ax = plt.subplots(1, 3, figsize=(12, 4))
        ax[0].loglog(self.r / R_sun, self.rho)
        ax[0].set_xlabel('r [Rsol]')
        ax[0].set_ylabel('density')
        ax[1].loglog(self.r / R_sun, self.m / MSol)
        ax[1].set_xlabel('r [Rsol]')
        ax[1].set_ylabel('mass [Msol]')
        ax[2].loglog(self.r / R_sun, self.P)
        ax[2].set_xlabel('r [m]')
        ax[2].set_ylabel('Pressure')
        # use sf in x axis

        plt.tight_layout()
        plt.show()

    def to_dataframe(self):
        return pd.DataFrame({'r': self.r, 'P': self.P, 'rho': self.rho, 'm': self.m})


star = Star.build(S0=2.5 * 1e14, rho0=1, gamma=5. / 3., m0=0)
star.plot_profile()
print(star.to_dataframe())


nan


/var/folders/qt/rxjvm_j566v9qn7g754s1v9hzb3p7f/T/ipykernel_71406/3637488407.py:17: RuntimeWarning: invalid value encountered in power
  return np.power(Pi / S, 1. / gamma)


IndexError: index 1000 is out of bounds for axis 0 with size 1000

### Lane emden

    The main sequence profile for a Lane-emden polytropic star (given the mass, radius and n)

    Profile consists of the following:
     - central pressure, density, the constant of proportionality K and the scale length ($r_{n}$).

    The formulas listed are derived and discussed in this
    [book](https://link.springer.com/book/10.1007/978-1-4419-9110-2):
    $$ P_c = \frac{8.952e+14}{(n+1)(\theta'_{n})^2_{\xi_1}}\left(\frac{M}{M_\odot}\right)^2\left(\frac{R}{R_\odot}\right)^{-4}dyne.cm^{-2} $$
    $$K = \frac{G}{n+1}M^{1-1/n}R^{-1+3/n}\left(\frac{4\pi}{\xi_1^{n+1}(-\theta'_n\Bigr|_{\xi_1})^{n-1}}\right)^{1/n}$$
    $$\rho_c = \left(\frac{P_c}{K}\right)^{\frac{n}{n+1}} g/cc$$
    $$r_n = \sqrt{\frac{(n+1)P_c}{4\pi G\rho_c^2}}$$

    We can use the dimensionless definitions used to derive the Lane-Emden equation to determine the density and pressure profiles
    $$P = P_{c} \theta_{n}^{n+1}$$
    $$ \rho = \rho_{c}\theta_{n}^{n}$$

    To get the mass profile in terms of the polytrope, we separate variables in the mass continuity equation and
    substitute the polytropic properties where applicable
    $$\frac{dM}{dr} = 4\pi r^{2}\rho$$
    $$dM = 4\pi r^{2} \rho dr$$
    $$M(r) = 4\pi \int_{0}^{R} r^{2}\rho dr$$
    $$M(\xi) = 4\pi r_n^3 \rho_c (-\xi^2\theta'_n\Bigr|_\xi)$$